In [1]:
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-rfe1b_b8
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-rfe1b_b8
  Resolved https://github.com/ibm-granite-community/utils to commit 1514191fbbc4605ed4fdfdcb448f2ee41477058f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
    model_kwargs={"max_tokens":1024, "temperature":0.2},
)

REPLICATE_API_TOKEN loaded from Google Colab secret.


In [4]:
def zeroshot_prompt_diet(data):
    return f"""
    Saya pengguna aplikasi diet bernama Kaloriku. Hari ini saya telah mengonsumsi {data['kalori_masuk']}kkal dan membakar {data['kalori_terbakar']}kkal melalui aktivitas.
    Target saya adalah defisit {data['target_defisit']}kkal.
    Tolong berikan saran makanan yang harus saya kurangi atau aktivitas tambahan yang bisa saya lakukan agar target defisit tercapai.
    Berikan jawaban ringkas dan mudah dipahami.
    """

# Panggil fungsi dan kirim ke Granite
prompt = zeroshot_prompt_diet({
    'kalori_masuk': 1800,
    'kalori_terbakar': 300,
    'target_defisit': 500
})

# Panggil AI
result = model.invoke(prompt)
print(result)



Saya memahami bahwa Anda menggunakan aplikasi diet bernama Kaloriku dan ingin mencapai defisit kalori sebesar 500 kkal. Anda telah mengonsumsi 1800 kcal dan membakar 300 kcal melalui aktivitas. 

Untuk mencapai target defisit 500 kcal, Anda perlu mengurangi kalori masukan atau meningkatkan kalori terbakar sebesar 500 kcal. Berikut adalah beberapa saran untuk Anda:

1. **Mengurangi Kalori Masukan:**
   - **Makanan yang dapat Anda kurangi:**
     - Makanan yang tinggi kalori dan rendah nutrisi, seperti makanan junk food, makanan fried, dan minuman beralkohol.
     - Snack makanan ringan seperti kue, biskuit, dan chips.
     - Makanan yang tinggi karbohidrat dan rendah protein, seperti nasi putih bersih tanpa tambahan protein lainnya.
   - **Sajikan makanan dengan lebih sehat:**
     - Pilihan makanan lebih sehat yang dapat Anda pilih adalah grill, steamed, atau rebusan buatan sendiri.
     - Tambahkan protein lebih banyak dalam setiap makanan, misalnya daging, ayam, sotong, atau tofu.
  

In [8]:
# Fungsi pembuat prompt dan pemanggil model Granite

def zeroshot_prompt(context, question, book_titles):
    """
    Membuat prompt yang akan dikirim ke model Granite.
    """
    titles = ", ".join(book_titles)
    prompt = f"""
    Kamu adalah ahli nutrisi dan pelatih kebugaran profesional.
    Tugasmu adalah memberikan saran berbasis data diet harian berikut ini:

    {context}
    {question}

    Tambahkan referensi dari: {titles if titles else 'tidak ada'}.
    Jawaban harus ringkas, langsung, dan mudah dimengerti pengguna awam.
    """
    return prompt


def get_answer_using_zeroshot(context, question, book_titles):
    """
    Mengirim prompt ke model Granite dan mengembalikan hasilnya.
    """
    prompt = zeroshot_prompt(context, question, book_titles)
    result = model.invoke(prompt)
    return result


In [9]:
context = "Saya adalah pengguna aplikasi diet harian Kaloriku."
question = "Hari ini saya makan 1800kkal dan olahraga 300kkal. Target saya defisit 500kkal. Apa saran makanan atau aktivitas selanjutnya?"
book_titles = []  # kosong karena tidak relevan dalam konteks ini

# Minta Granite memberikan saran diet
result = get_answer_using_zeroshot(context, question, book_titles)
print(f"Saran AI Granite:\n{result}")


Saran AI Granite:
Selamat menggunakan aplikasi diet harian! Berdasarkan informasi yang diberikan, Anda mencapai defisit kalori target Anda (500 kkal) dengan makanan 1800 kkal dan olahraga 300 kkal, yang menghasilkan defisit sebanyak 1000 kkal (1800 - 300 = 1500, dan 1500 - 500 = 1000).

Saran untuk makanan dan aktivitas:

1. **Makanan**: Tetap konsumsi makanan Anda dengan total kalori 1800, tetapi pastikan makanan Anda terdistribusi seimbang antara macam-macam macronutrient (karbohidrat, protein, dan lemak). Coba untuk mencakup:

   - Sumber karbohidrat sehat seperti buah, sayuran, dan grain whole.
   - Sumber protein seperti daging, ikan, tofu, atau tempe.
   - Makanan yang kaya dengan fiber untuk membantu rasa satisfied dan keseimbangan hormon leptin dan ghrelin.

2. **Aktivitas**: Untuk mendukung tujuan kebugaran Anda, pertimbangkan untuk menambahkan aktivitas fisik lebih intensif secara berkala. Misalnya:

   - Lakukan olahraga cardio seperti joging, bersiul, atau berolak-olah sela